In [13]:
import numpy as np

class Unknown:
    def __init__(self) -> None:
        pass
    def __repr__(self) -> str:
        return "?"
    def __add__(self,_):
        return self
    def __sub__(self,_):
        return self

axis = 0

a = np.array([3,4,7,2,5])

a1 = np.roll(a,1)

edge_shape = list(a.shape)
edge_shape[axis]+=1

diff = np.full(tuple(edge_shape),Unknown(),dtype="object")


diff[1:]+=a
diff[1:]-=a1

diff

array([?, ?, ?, ?, ?, ?], dtype=object)

In [14]:
class Unknown:
    def __init__(self):
        pass

    def __repr__(self):
        return "?"

    def __neg__(self):
        return self
        
    def __add__(self,_):
        return self
    
    def __sub__(self,_):
        return self
        
    def __mul__(self,_):
        return self

    def __truediv__(self,_):
        return self
   
    def __pow__(self,_):
        return self
    
    def __radd__(self,_):
        return self
    
    def __rsub__(self,_):
        return self
    
    def __rmul__(self,_):
        return self
    
    def __rtruediv__(self,_):
        raise self
    
    def __rpow__(self,_):
        raise self

    def __lt__(self,_):
        return False

In [40]:
import time

n_times = 10000
shape = (100,100)

A_obj = np.ones(shape,dtype="object")
A = np.full(shape,np.nan)

def timeit(f,name):
    start = time.time()
    f()
    end = time.time()
    print(f"{name}: {end-start}")

def obj_calc():
    for _ in range(n_times):
        A_obj*0



def float_calc():
    for _ in range(n_times):
        A*0



timeit(obj_calc,"object")

timeit(float_calc,"float")


object: 0.5840003490447998
float: 0.02509903907775879


In [47]:
import numpy as np

x = np.array([[np.nan, 3.14, np.nan]],dtype="object")
mx = np.ma.masked_invalid(x)

mx.mask

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''